<a href="https://colab.research.google.com/github/Kabongosalomon/Word-Embedding-Investigation/blob/master/%5BTransformer%2C%20BERT%5D%20An%20empirical%20investigation%20into%20the%20properties%20of%20standard%20word%20embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://raw.githubusercontent.com/Kabongosalomon/Data-Science-With-Jeff-Sander/master/aims-za-logo.jpg" width="100" alt="cognitiveclass.ai logo" />





# IMdb Sentiment Analysis Task

## Importing usefull packages 

In [1]:
import torch

import tensorflow as tf
!pip install transformers
import transformers as ppb

import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

import nltk
nltk.download('punkt')
nltk.download('stopwords')

import numpy as np

import matplotlib.pyplot as plt

import os

import numpy as np

import string

!pip install ipdb
import ipdb # deb

# from gensim.models.keyedvectors import KeyedVectors

# Spliting data
from sklearn.model_selection import train_test_split

from sklearn import metrics # For RUC

from nltk.stem import PorterStemmer

import pandas as pd
import re
import seaborn as sns

# from google.colab import files

from IPython import display

import logging
logging.getLogger('googleapiclient.discovery_cache').setLevel(logging.ERROR)

     |████████████████████████████████| 573kB 3.3MB/s 
     |████████████████████████████████| 890kB 37.4MB/s 
     |████████████████████████████████| 1.0MB 32.2MB/s 
     |████████████████████████████████| 3.7MB 32.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=3ddf8deba1e2afdf0380a673b36577b0328db9b1def95cda71ee272d88a5f403
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built sacremoses
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
  Created wheel for ipdb: filename=ipdb-0.13.2-cp36-none-any.whl size=10522 sha256=3276e0a27dd19fb8924c73cbda76fa097034f67ee295068816eb10facdec7939
  Stored in directory: /root/.cache/pip/wheels/60/c2/15/793365e3c9318c46ba914263740d90f1fe67f544b979141ce4
Success

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
link = "https://drive.google.com/file/d/1smGRs2g2HoI6VSvonoZmWKzXOP6uPUaW/view?usp=sharing"

_, id_t = link.split('d/')

id = id_t.split('/')[0]
print (id) # Verify that you have everything after '='

# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


file_id = id
downloaded = drive.CreateFile({'id':file_id})
downloaded.FetchMetadata(fetch_all=True)
downloaded.GetContentFile(downloaded.metadata['title'])

1smGRs2g2HoI6VSvonoZmWKzXOP6uPUaW


In [0]:
!ls

aclImdb.zip  adc.json  sample_data


In [0]:
!unzip -qq aclImdb.zip

In [0]:
!ls

aclImdb  aclImdb.zip  adc.json	__MACOSX  sample_data


In [0]:
# imdb_dir = './data/aclImdb'
imdb_dir = './aclImdb'

# Reading in the training folder
train_dir = os.path.join(imdb_dir, 'train')
texts_tr_ = []
labels_tr = []

for label_type in ['neg', 'pos']:
    dir_name = os.path.join(train_dir, label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname), encoding="utf8")
            texts_tr_.append(f.read())
            f.close()
            if label_type == 'neg':
                labels_tr.append(0)
            else:
                labels_tr.append(1)

# Reading in the testing folder
train_dir = os.path.join(imdb_dir, 'test')
texts_tst_ = []
labels_tst = []

for label_type in ['neg', 'pos']:
    dir_name = os.path.join(train_dir, label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname), encoding="utf8")
            texts_tst_.append(f.read())
            f.close()
            if label_type == 'neg':
                labels_tst.append(0)
            else:
                labels_tst.append(1)

In [0]:
# Make sure that we have only 1 and 2 in the label 
(np.unique(labels_tr), np.unique(labels_tst))

(array([0, 1]), array([0, 1]))

In [0]:
len(labels_tr)

25000

In [0]:
# Looking at 2 examples
print(texts_tr_[1])
print(labels_tr[1])

print(texts_tst_[-10])
print(labels_tst[-10])

There are movies that are awful, and there are movies that are so awful they are deemed long-forgotten and unwatchable. Also, lots of violence and bad stuff (not just cheesy stuff; you know what I mean) add to the mix as well. What is the result of bad movies with such raunchy content? Why, "Final Justice," of course! <br /><br />Remember "Mitchell?" Joe Don Baker was the star of that movie, and that was riffed by Joel and the Bots on "Mystery Science Theater 3000." Now this time, with Mike taking Joel's place on the Satellite of Love (but with the same bots), that trio got to make fun of MST3K's second Joe Don Baker movie, "Final Justice." Of course, much of the naughty stuff that I mentioned was removed for television release, but still, I want to watch that episode (and "Mitchell" as well), because what does Joe Don "hate" the most? Why, none other than "Mystery Science Theater 3000!" <br /><br />P.S. If you have a Big Lots nearby, check that store for the uncut tape! LOL That happe

### Getting started

#### Data
We will try to solve the [Large Movie Review Dataset v1.0](http://ai.stanford.edu/~amaas/data/sentiment/) task from Mass et al. The dataset consists of IMDB movie reviews labeled by positivity from 1 to 10. The task is to label the reviews as **negative** or **positive**.

In [0]:
# Load all files from a directory in a DataFrame.
def load_directory_data(directory):
  data = {}
  data["sentence"] = []
#   data["sentiment"] = []
  for file_path in os.listdir(directory):
    with tf.io.gfile.GFile(os.path.join(directory, file_path), "r") as f:
      data["sentence"].append(f.read())
#       data["sentiment"].append(re.match("\d+_(\d+)\.txt", file_path).group(1))#; ipdb.set_trace()
  return pd.DataFrame.from_dict(data)

# Merge positive and negative examples, add a polarity column and shuffle.
def load_dataset(directory):
  pos_df = load_directory_data(os.path.join(directory, "pos"))
  neg_df = load_directory_data(os.path.join(directory, "neg"))
  pos_df["sentiment"] = 1
  neg_df["sentiment"] = 0
  return pd.concat([pos_df, neg_df]).sample(frac=1).reset_index(drop=True)

# Download and process the dataset files.
def download_and_load_datasets(force_download=False):
  dataset = tf.keras.utils.get_file(
      fname="aclImdb.tar.gz", 
      origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
      extract=True)
  
  train_df = load_dataset(os.path.join(os.path.dirname(dataset), 
                                       "aclImdb", "train"))
  test_df = load_dataset(os.path.join(os.path.dirname(dataset), 
                                      "aclImdb", "test"))
  
  return train_df, test_df


train_df, test_df = download_and_load_datasets()

84131840/84125825 [==============================] - 9s 0us/step


In [0]:
# Double-check that we've done the right thing.
print("Training examples summary:")
display.display(train_df.describe())
display.display(train_df.head())
print("\n#############################\n")
print("Validation examples summary:")
display.display(test_df.describe())
display.display(test_df.head())

Training examples summary:


,sentiment
count,25000.00000
mean,0.50000
std,0.50001
min,0.00000
25%,0.00000
50%,0.50000
75%,1.00000
max,1.00000


,sentence,sentiment
0,I've seen this film on Sky Cinema not too long...,1
1,Had placed this on my TIVO for a rainy day due...,1
2,I admire 'Kissing on the Mouth' for its frankn...,0
3,The premise of this movie is revealed on the D...,0
4,Found this one in the video store and rented i...,1



#############################

Validation examples summary:


,sentiment
count,25000.00000
mean,0.50000
std,0.50001
min,0.00000
25%,0.00000
50%,0.50000
75%,1.00000
max,1.00000


,sentence,sentiment
0,The Plot: A group of young people with ridicul...,0
1,"If you take the films, Mad Max, Beyond Thunder...",0
2,"Another movie from Swedish hillbilly country, ...",0
3,It was easy to dismiss this film as hyperbole ...,1
4,Fantastic movie! One of the best film noir mov...,1


In [0]:
train_df['sentence'].apply(lambda x : len(x))

0         994
1         882
2        1800
3        1231
4         268
         ... 
24995    1141
24996    1738
24997    3372
24998    2581
24999    1473
Name: sentence, Length: 25000, dtype: int64

In [0]:
# mean lengh of review 
train_df['sentence'].apply(lambda x : len(x)).mean()

1325.06964

In [0]:
train_df['sentence'][0]

"I've seen this film on Sky Cinema not too long ago.. I must admit, it was a really good Western which features 2 of the big names.. On one side, there's Charlton Heston, playing the infamous and retired lawman Samuel Burgade. On the other.. The late James Coburn playing the villainous Zach Provo.. seeking revenge on Burgade no matter what the cost..!<br /><br />The good thing about this film was there was some really good characters.. Most of the actors played it out really well.. Especially James Coburn, who I find that he was really mean in this film.. But that how it was..<br /><br />Christopher Mitchum, who I've seen everywhere in other films.. Playing Hal Brickman.. I felt his character was left out in the cold, but he manage to get himself back in by teaming up with Burgade, to bring down Provo's posse's!<br /><br />All in all, it was a great film.. Very good to watch.. Great score from the late Jerry Goldsmith..<br /><br />Wonderful piece of Western persona..! 8 out of 10."

# Obtain Pre-trained BERT Model

In [0]:
# !pip install torchtext

In [0]:


# # define VGG16 model
# # VGG16 = models.vgg16(pretrained=True)

# DistilBERT = models.transformers()

# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# check if CUDA is available
use_cuda = torch.cuda.is_available()



# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

# move model to GPU if CUDA is available
if use_cuda:
    model = model.cuda()
#     tokenizer = tokenizer.cuda()

In [0]:
class LossPrettifier(object):
    
    STYLE = {
        'green' : '\033[32m',
        'red'   : '\033[91m', 
        'bold'  : '\033[1m', 
    }
    STYLE_END = '\033[0m'
    
    def __init__(self, show_percentage=False):
        
        self.show_percentage = show_percentage
        self.color_up = 'green'
        self.color_down = 'red'
        self.loss_terms = {}
    
    def __call__(self, epoch=None, **kwargs):
        
        if epoch is not None:
            print_string = f'Epoch {epoch: 5d} '
        else:
            print_string = ''

        for key, value in kwargs.items():
            
            pre_value = self.loss_terms.get(key, value)
            
            if value > pre_value:
                indicator  = '▲'
                show_color = self.STYLE[self.color_up]
            elif value == pre_value:
                indicator  = ''
                show_color = ''
            else:
                indicator  = '▼'
                show_color = self.STYLE[self.color_down]
            
            if self.show_percentage:
                show_value = 0 if pre_value == 0 \
                             else (value - pre_value) / float(pre_value)
                key_string = f'| {key}: {show_color}{value:3.4f}({show_value:+3.4%}) {indicator}'
            else: 
                key_string = f'| {key}: {show_color}{value:.4f} {indicator}'
            
            # Trim some long outputs
            key_string_part = key_string[:32]
            print_string += key_string_part+f'{self.STYLE_END}\t'
            
            self.loss_terms[key] = value
            
        print(print_string)
reporter = LossPrettifier(show_percentage=True)

In [0]:
def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    
    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        
        ###################
        # train the model #
        ###################
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            ## find the loss and update the model parameters accordingly
            ## record the average training loss, using something like
            ## train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data)
            # calculate the batch loss
            loss = criterion(output, target)
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
            # update training loss
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss)) 
            
        ######################    
        # validate the model #
        ######################
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            ## update the average validation loss
            output = model(data)
            # calculate the batch loss
            loss = criterion(output, target)
            # update average validation loss 
            valid_loss +=((1 / (batch_idx + 1)) * (loss.data - valid_loss))
            
            
        # print training/validation statistics 
        
        reporter(epoch=epoch, LossA = train_loss, LossB = valid_loss)
        
        ## TODO: save the model if validation loss has decreased
        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_loss_min,
            valid_loss))
            torch.save(model.state_dict(), save_path)
            valid_loss_min = valid_loss
            
    # return trained model
    return model

# loaders_scratch = {'train' : train_loader, 'valid': valid_loader, 'test' :test_loader}

# # train the model
# model_scratch = train(15, loaders_scratch, model_scratch, optimizer_scratch, 
#                       criterion_scratch, use_cuda, 'model_scratch.pt')

In [0]:
def test(loaders, model, criterion, use_cuda):

    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    model.eval()
    for batch_idx, (data, target) in enumerate(loaders['train']):
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))

# call test function    
# test(loaders_scratch, model_scratch, criterion_scratch, use_cuda)

In [0]:
!ls

sample_data


Right now, the variable `model` holds a pretrained distilBERT model -- a version of BERT that is smaller, but much faster and requiring a lot less memory.

## Model #1: Preparing the Dataset
Before we can hand our sentences to BERT, we need to so some minimal processing to put them in the format it requires.

### Tokenization
Our first step is to tokenize the sentences -- break them up into word and subwords in the format BERT is comfortable with.

Unfortunatly the bert tokenizer can only encode sequence lenght less or equal to 512

In [0]:
tokenized = train_df['sentence'][:1000].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=50))) 
# tokenized_test = test_df['sentence'][:500].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=300)))

<img src="https://jalammar.github.io/images/distilBERT/bert-distilbert-tokenization-2-token-ids.png" />

### Padding
After tokenization, `tokenized` is a list of sentences -- each sentences is represented as a list of tokens. We want BERT to process our examples all at once (as one batch). It's just faster that way. For that reason, we need to pad all lists to the same size, so we can represent the input as one 2-d array, rather than a list of lists (of different lengths).

In [0]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

# max_len = 0
# for i in tokenized_train.values:
#     if len(i) > max_len:
#         max_len = len(i)

# padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized_train.values])

Our dataset is now in the `padded` variable, we can view its dimensions below:

In [0]:
np.array(padded).shape

(1000, 50)

### Masking
If we directly send `padded` to BERT, that would slightly confuse it. We need to create another variable to tell it to ignore (mask) the padding we've added when it's processing its input. That's what attention_mask is:

In [0]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(1000, 50)

## Model #1: And Now, Deep Learning!
Now that we have our model and inputs ready, let's run our model!

<img src="http://127.0.0.1:4000/images/distilBERT/bert-distilbert-tutorial-sentence-embedding.png" />

The `model()` function runs our sentences through BERT. The results of the processing will be returned into `last_hidden_states`.

In [0]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

if use_cuda:
    input_ids = input_ids.cuda()
    attention_mask = attention_mask.cuda()


with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

Let's slice only the part of the output that we need. That is the output corresponding the first token of each sentence. The way BERT does sentence classification, is that it adds a token called `[CLS]` (for classification) at the beginning of every sentence. The output corresponding to that token can be thought of as an embedding for the entire sentence.

<img src="https://jalammar.github.io/images/distilBERT/bert-output-tensor-selection.png" />

We'll save those in the `features` variable, as they'll serve as the features to our logitics regression model.

In [0]:
features = last_hidden_states[0][:,0,:].cpu().numpy()

The labels indicating which sentence is positive and negative now go into the `labels` variable

In [0]:
batch_1 = train_df[:1000]

In [0]:
batch_1.head()

,sentence,sentiment
0,I've seen this film on Sky Cinema not too long...,1
1,Had placed this on my TIVO for a rainy day due...,1
2,I admire 'Kissing on the Mouth' for its frankn...,0
3,The premise of this movie is revealed on the D...,0
4,Found this one in the video store and rented i...,1


In [0]:
labels = batch_1['sentiment']

## Model #2: Train/Test Split
Let's now split our datset into a training set and testing set (even though we're using 2,000 sentences from the SST2 training set).

<img src="https://jalammar.github.io/images/distilBERT/sentiment-classifier-1.png" />

Under the hood, the model is actually made up of two model.

* DistilBERT processes the sentence and passes along some information it extracted from it on to the next model. DistilBERT is a smaller version of BERT developed and open sourced by the team at HuggingFace. It’s a lighter and faster version of BERT that roughly matches its performance.
* The next model, a basic Logistic Regression model from scikit learn will take in the result of DistilBERT’s processing, and classify the sentence as either positive or negative (1 or 0, respectively).

The data we pass between the two models is a vector of size 768. We can think of this of vector as an embedding for the sentence that we can use for classification.

In [0]:
train_features, test_features, train_labels, test_labels = train_test_split(features, labels)

In [0]:
features.shape

(1000, 768)

In [0]:
labels.shape

(1000,)

In [0]:
last_hidden_states

(tensor([[[ 0.1768, -0.0254, -0.1115,  ...,  0.0434,  0.5822,  0.3047],
          [ 0.7705, -0.0778,  0.0697,  ...,  0.2148,  0.8541,  0.3128],
          [ 0.8751,  0.3873, -0.2589,  ...,  0.0973, -0.4155, -0.2554],
          ...,
          [-0.4498, -0.1418, -0.0336,  ...,  0.2337,  0.5375, -0.3464],
          [-0.1297, -0.1688, -0.3179,  ...,  0.3928,  0.1677, -0.7424],
          [ 0.7855,  0.4907, -0.1765,  ...,  0.2381, -0.3008, -0.1600]],
 
         [[ 0.0446, -0.2627,  0.1647,  ..., -0.2541,  0.3769,  0.1559],
          [ 0.6444, -0.1880, -0.1071,  ..., -0.0109,  0.6716,  0.2098],
          [ 0.2220, -0.1627,  0.0870,  ...,  0.0194, -0.0905,  0.1047],
          ...,
          [-0.0930, -0.4018,  0.2851,  ..., -0.2901, -0.0851,  0.2510],
          [-0.1907, -0.2176,  0.0833,  ...,  0.0604,  0.0753, -0.2888],
          [ 0.4157,  0.2264,  0.6237,  ...,  0.0103,  0.1531, -0.3304]],
 
         [[ 0.1874, -0.0134, -0.2633,  ..., -0.1330,  0.4747,  0.3184],
          [ 0.5238,  0.0329,

In [0]:
import os
from torchvision import datasets


## TODO: Specify data loaders
num_workers = 0
# how many samples per batch to load
batch_size = 20


# define training, valid and test data directories
data_dir = './aclImdb/'
train_dir = os.path.join(data_dir, 'train/')
valid_dir = os.path.join(data_dir, 'valid/')
test_dir = os.path.join(data_dir, 'test/')

# data_transform = transforms.Compose([
#     transforms.RandomResizedCrop(224),
#     transforms.RandomHorizontalFlip(), # randomly flip and rotate
# #     transforms.RandomRotation(10),
#     transforms.ToTensor(),
#     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
#     ])

train_data = datasets.ImageFolder(train_dir)
# valid_data = datasets.ImageFolder(valid_dir)
test_data = datasets.ImageFolder(test_dir)


train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
    num_workers=num_workers)
# valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=batch_size, 
#     num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
    num_workers=num_workers)

RuntimeError: ignored

### (IMPLEMENTATION) Model Architecture

Use transfer learning to create a CNN to classify dog breed.  Use the code cell below, and save your initialized model as the variable `model_transfer`.

In [0]:
import torch.nn as nn
import torch.nn.functional as F


class Net_from_Bert(nn.Module):
    ### TODO: choose an architecture, and complete the class
    def __init__(self):
        super(Net_from_Bert, self).__init__()
        self.fc = nn.Linear(features.shape[1], 2)
    
    def forward(self, x):
        x = self.fc(x)
        return x


model = Net_from_Bert()

# move tensors to GPU if CUDA is available
if use_cuda:
    model.cuda()

__Question 5:__ Outline the steps you took to get to your final CNN architecture and your reasoning at each step.  Describe why you think the architecture is suitable for the current problem.

__Answer:__ 

- I used the pretrained vgg16 and just changed the last layer to output 133 classes.
- The reason behind using the vgg16 is the it's a pre-trained model on huge images and by freezing the weight we can take advantage of what the model learned from the original training data. 
- I thin kthis architecture is suitable for the dog breed problem because the vgg16 also was trained on imagenet and some of those have some caracteristic in common with my task. 

### (IMPLEMENTATION) Specify Loss Function and Optimizer

Use the next code cell to specify a [loss function](http://pytorch.org/docs/master/nn.html#loss-functions) and [optimizer](http://pytorch.org/docs/master/optim.html).  Save the chosen loss function as `criterion_transfer`, and the optimizer as `optimizer_transfer` below.

In [0]:
import torch.optim as optim

criterion_model = nn.CrossEntropyLoss()
optimizer_model = optim.SGD(model.parameters(), lr=0.001)

### (IMPLEMENTATION) Train and Validate the Model

Train and validate your model in the code cell below.  [Save the final model parameters](http://pytorch.org/docs/master/notes/serialization.html) at filepath `'model_transfer.pt'`.

In [0]:
loaders_transfer = {'train' : train_loader, 'valid': valid_loader, 'test' :test_loader}

In [0]:
# train the model
model_transfer = train(15, loaders_transfer, model, 
                       optimizer_transfer, criterion_transfer, use_cuda, 'model_transfer.pt')

In [0]:
# load the model that got the best validation accuracy (uncomment the line below)
model_transfer.load_state_dict(torch.load('model_transfer.pt'))

### (IMPLEMENTATION) Test the Model

Try out your model on the test dataset of dog images. Use the code cell below to calculate and print the test loss and accuracy.  Ensure that your test accuracy is greater than 60%.

In [0]:
test(loaders_transfer, model_transfer, criterion_transfer, use_cuda)

### (IMPLEMENTATION) Predict Dog Breed with the Model

Write a function that takes an image path as input and returns the dog breed (`Affenpinscher`, `Afghan hound`, etc) that is predicted by your model.  

In [0]:
path = '/data/dog_images/train/001.Affenpinscher/Affenpinscher_00001.jpg'
data_transfer = {'train' : train_data, 'valid': valid_data, 'test' :test_data}

In [0]:
### TODO: Write a function that takes a path to an image as input
### and returns the dog breed that is predicted by the model.

# list of class names by index, i.e. a name can be accessed like class_names[0]
class_names = [item[4:].replace("_", " ") for item in data_transfer['train'].classes]

def predict_breed_transfer(img_path):
    # load the image and return the predicted breed
    
    data_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(), # randomly flip and rotate
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])
    
    img = data_transform(Image.open(img_path))
    if use_cuda:
        img = img.cuda()
        
    # PyTorch pretrained models expect the Tensor dims to be (num input imgs, num color channels, height, width).
    # Currently however, we have (num color channels, height, width); let's fix this by inserting a new axis.
    img = img.unsqueeze(0)  # Insert the new axis at index 0 i.e. in front of the other axes/dims. 
    
    prediction = model_transfer(img)  # Returns a Tensor of shape (batch, num class labels)

    
    return class_names[prediction.data.cpu().numpy().argmax()]

In [0]:
!pip install transformers

     |████████████████████████████████| 573kB 42.0MB/s 
     |████████████████████████████████| 1.0MB 50.1MB/s 
     |████████████████████████████████| 3.7MB 50.5MB/s 
     |████████████████████████████████| 890kB 53.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=e9a76c26a24acfb4f7f7b812bf85d58c24ac9e7ccbda25a517933e1e2593dd59
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built sacremoses


In [0]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

In [0]:
import random

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [4]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [5]:
len(tokenizer.vocab)


30522

In [6]:
init_token = tokenizer.cls_token
eos_token = tokenizer.sep_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

print(init_token, eos_token, pad_token, unk_token)

[CLS] [SEP] [PAD] [UNK]


In [7]:
init_token_idx = tokenizer.cls_token_id
eos_token_idx = tokenizer.sep_token_id
pad_token_idx = tokenizer.pad_token_id
unk_token_idx = tokenizer.unk_token_id

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

101 102 0 100


Another thing we need to handle is that the model was trained on sequences with a defined maximum length - it does not know how to handle sequences longer than it has been trained on. We can get the maximum length of these input sizes by checking the max_model_input_sizes for the version of the transformer we want to use. In this case, it is 512 tokens.

In [8]:
max_input_length = tokenizer.max_model_input_sizes['bert-base-uncased']

print(max_input_length)

512


Previously we have used the spaCy tokenizer to tokenize our examples. However we now need to define a function that we will pass to our TEXT field that will handle all the tokenization for us. It will also cut down the number of tokens to a maximum length. Note that our maximum length is 2 less than the actual maximum length. This is because we need to append two tokens to each sequence, one to the start and one to the end.

In [0]:

def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence) 
    tokens = tokens[:max_input_length-2]
    return tokens


Now we define our fields. The transformer expects the batch dimension to be first, so we set batch_first = True. As we already have the vocabulary for our text, provided by the transformer we set use_vocab = False to tell torchtext that we'll be handling the vocabulary side of things. We pass our tokenize_and_cut function as the tokenizer. The preprocessing argument is a function that takes in the example after it has been tokenized, this is where we will convert the tokens to their indexes. Finally, we define the special tokens - making note that we are defining them to be their index value and not their string value, i.e. 100 instead of [UNK] This is because the sequences will already be converted into indexes.

We define the label field as before.

In [0]:

from torchtext import data

TEXT = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = tokenize_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = init_token_idx,
                  eos_token = eos_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)

LABEL = data.LabelField(dtype = torch.float)

In [11]:
from torchtext import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

aclImdb_v1.tar.gz:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:02<00:00, 36.0MB/s]


In [12]:
print(f"Number of training examples: {len(train_data)}")
print(f"Number of validation examples: {len(valid_data)}")
print(f"Number of testing examples: {len(test_data)}")

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


We can check an example and ensure that the text has already been numericalized.

In [0]:
print(vars(train_data.examples[6]))

{'text': [1999, 8741, 1997, 2383, 2070, 10990, 1006, 1998, 15236, 1007, 10071, 1010, 1050, 19279, 2078, 2074, 2196, 4152, 2183, 1012, 2045, 2024, 20728, 6242, 1010, 6045, 9231, 9916, 1010, 3742, 5920, 2015, 1010, 4130, 16886, 2007, 2303, 3033, 1010, 1998, 2035, 11901, 1997, 4438, 6547, 1013, 5469, 5019, 1010, 2021, 2027, 1005, 2128, 25338, 2007, 3668, 6993, 1997, 5760, 13080, 1012, 7955, 1999, 1996, 3185, 3504, 11471, 1012, 2023, 2003, 1037, 9467, 2138, 2116, 1997, 1996, 10071, 2052, 2022, 2641, 15236, 2012, 1996, 2051, 2023, 2001, 6361, 1012, 1026, 7987, 1013, 1028, 1026, 7987, 1013, 1028, 5587, 2000, 2023, 1996, 1000, 2205, 1011, 5372, 1000, 28101, 3494, 1998, 2017, 2514, 2066, 2017, 1005, 2310, 10070, 2046, 1037, 20052, 9106, 3185, 1012, 1026, 7987, 1013, 1028, 1026, 7987, 1013, 1028, 2633, 1010, 1996, 16434, 2003, 2200, 6623, 1012, 2045, 2024, 7167, 1997, 6695, 2005, 3376, 7171, 1997, 1997, 1996, 10833, 1010, 2030, 3375, 7171, 1997, 2619, 2108, 2766, 2046, 1037, 4121, 28698, 1010, 

We can use the convert_ids_to_tokens to transform these indexes back into readable tokens.

In [0]:
tokens = tokenizer.convert_ids_to_tokens(vars(train_data.examples[6])['text'])

print(tokens)

['in', 'spite', 'of', 'having', 'some', 'exciting', '(', 'and', 'daring', ')', 'sequences', ',', 'n', '##bt', '##n', 'just', 'never', 'gets', 'going', '.', 'there', 'are', 'exploding', 'boats', ',', 'hat', 'pin', 'murders', ',', 'mass', 'suicide', '##s', ',', 'path', '##ologists', 'with', 'body', 'parts', ',', 'and', 'all', 'sorts', 'of', 'classic', 'mystery', '/', 'horror', 'scenes', ',', 'but', 'they', "'", 're', 'interspersed', 'with', 'extended', 'periods', 'of', 'pure', 'exposition', '.', 'everybody', 'in', 'the', 'movie', 'looks', 'bored', '.', 'this', 'is', 'a', 'shame', 'because', 'many', 'of', 'the', 'sequences', 'would', 'be', 'considered', 'daring', 'at', 'the', 'time', 'this', 'was', 'filmed', '.', '<', 'br', '/', '>', '<', 'br', '/', '>', 'add', 'to', 'this', 'the', '"', 'too', '-', 'proper', '"', 'brit', 'characters', 'and', 'you', 'feel', 'like', 'you', "'", 've', 'drifted', 'into', 'a', 'sherlock', 'holmes', 'movie', '.', '<', 'br', '/', '>', '<', 'br', '/', '>', 'final

Although we've handled the vocabulary for the text, we still need to build the vocabulary for the labels.

In [0]:
LABEL.build_vocab(train_data)


In [0]:
BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)

In [15]:
from transformers import BertTokenizer, BertModel

bert = BertModel.from_pretrained('bert-base-uncased')

In [0]:
import torch.nn as nn

class BERT_FNN_Sentiment(nn.Module):
    def __init__(self,
                 bert,
                 hidden_dim,
                 number_classes):
        
        super().__init__()
        
        self.bert = bert
        
        embedding_dim = bert.config.to_dict()['hidden_size']
        
        self.fc = nn.Linear(393216, number_classes) # got this error from pytorch error :)
                
        # self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [batch size, sent len]
                
        with torch.no_grad():
            embedded = self.bert(text)[0] 
        # ipdb.set_trace()  
        #embedded = [batch size, sent len, emb dim]
        
        batch_size = embedded.shape[0]
        
        # x = self.linear(x.view(batch_size, -1))

        output = self.fc(embedded.view(batch_size, -1))
        
        
        return output

In [0]:
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = BERT_FNN_Sentiment(bert,
                         HIDDEN_DIM,
                         OUTPUT_DIM)


We can check how many parameters the model has. Our standard models have under 5M, but this one has 112M! Luckily, 110M of these parameters are from the transformer and we will not be training those.

In [104]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 393,217 trainable parameters


In order to freeze paramers (not train them) we need to set their requires_grad attribute to False. To do this, we simply loop through all of the named_parameters in our model and if they're a part of the bert transformer model, we set requires_grad = False.

In [0]:
for name, param in model.named_parameters():                
    if name.startswith('bert'):
        param.requires_grad = False

In [106]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 393,217 trainable parameters


We can double check the names of the trainable parameters, ensuring they make sense. As we can see, they are all the parameters of tthe linear layer (out).

In [107]:
for name, param in model.named_parameters():                
    if param.requires_grad:
        print(name)

fc.weight
fc.bias


In [0]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [0]:
criterion = nn.BCEWithLogitsLoss()

In [0]:
model = model.to(device)
criterion = criterion.to(device)

In [0]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [0]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [0]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
        
    end_time = time.time()
        
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut6-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

We'll load up the parameters that gave us the best validation loss and try these on the test set - which gives us our best results so far!

In [0]:

model.load_state_dict(torch.load('tut6-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

## Inference
We'll then use the model to test the sentiment of some sequences. We tokenize the input sequence, trim it down to the maximum length, add the special tokens to either side, convert it to a tensor, add a fake batch dimension and then pass it through our model.

In [0]:

def predict_sentiment(model, tokenizer, sentence):
    model.eval()
    tokens = tokenizer.tokenize(sentence)
    tokens = tokens[:max_input_length-2]
    indexed = [init_token_idx] + tokenizer.convert_tokens_to_ids(tokens) + [eos_token_idx]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [0]:
predict_sentiment(model, tokenizer, "This film is terrible")


In [0]:
predict_sentiment(model, tokenizer, "This film is great")


---

# <h2>About the Authors:</h2> 

<a href="https://salomonkabongo.wixsite.com/datascientist">Salomon Kabongo KABENAMUALU</a>, Master degree student at <a href="https://aims.ac.za/">the African Institute for mathematical SCiences (AIMS South Africa)</a> his research focused on the use machine learning technique in the field of Natural Language Processing.

References : <a href="https://machinelearningmastery.com/deep-learning-bag-of-words-model-sentiment-analysis/"> How to Develop a Deep Learning Bag-of-Words Model for Predicting Movie Review Sentiment</a>, <a href="https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/6%20-%20Transformers%20for%20Sentiment%20Analysis.ipynb"> Github repo pytorch-sentiment-analysis</a>

Copyright &copy; 2019. This notebook and its source code are released under the terms of the <a href="https://www.apache.org/licenses/LICENSE-2.0">Apache License 2.0</a>.